# Tutorial for using PMO database

We illustrate briefly the main functionality of the package `PMO.jl` dedicated to Polynomial & Moment Optimization problems. 

## Polynomial data

In [1]:
using PMO, DynamicPolynomials

X = @polyvar x y

o = x^2*y^2+x^4-y^3
g1  = x^2 + Float64(pi)*y^2 -2
g2 = x

h1 = 2*y^2-y


F  = PMO.polynomial((o,"inf"),
                    (g1,"<=0"),
                    (g2,">=0"),
                    (h1,"=0"))

F["doc"]= 
"""
A first polynomial example with two sign constraints and one equality constraint.
"""

"A first polynomial example with two sign constraints and one equality constraint.\n"

This object of type `PMOData` can be printed and saved in json format as follows:

In [2]:
PMO.json(F)
PMO.save("tmp.json",F)

{
  "type": "polynomial",
  "variables": [
    "x",
    "y"
  ],
  "nvar": 2,
  "constraints": [
    {
      "set": "<=0",
      "polynomial": {
        "coeftype": "Float64",
        "terms": [
          [1.0,[2],[1]],
          [3.141592653589793,[2],[2]],
          [-2.0]
        ]
      }
    },
    {
      "set": ">=0",
      "polynomial": {
        "coeftype": "Int64",
        "terms": [
          [1,[1],[1]]
        ]
      }
    },
    {
      "set": "=0",
      "polynomial": {
        "coeftype": "Int64",
        "terms": [
          [2,[2],[2]],
          [-1,[1],[2]]
        ]
      }
    }
  ],
  "objective": {
    "set": "inf",
    "polynomial": {
      "coeftype": "Int64",
      "terms": [
        [1,[4],[1]],
        [1,[2,2]],
        [-1,[3],[2]]
      ]
    }
  },
  "version": "0.0.1",
  "uuid": "dedf3ace-0a05-11eb-172f-49332467ce3a",
  "doc": "A first polynomial example with two sign constraints and one equality constraint.\n"
}


We read the data saved in the file `tmp.json` and built the corresponding data `G` of type `PMOData`, which `json` format is the same as the one of `F`.

In [3]:
G  = PMO.read("tmp.json")
PMO.json(G)

{
  "type": "polynomial",
  "variables": [
    "x",
    "y"
  ],
  "nvar": 2,
  "constraints": [
    {
      "set": "<=0",
      "polynomial": {
        "coeftype": "Float64",
        "terms": [
          [1.0,[2],[1]],
          [3.141592653589793,[2],[2]],
          [-2.0]
        ]
      }
    },
    {
      "set": ">=0",
      "polynomial": {
        "coeftype": "Int64",
        "terms": [
          [1,[1],[1]]
        ]
      }
    },
    {
      "set": "=0",
      "polynomial": {
        "coeftype": "Int64",
        "terms": [
          [2,[2],[2]],
          [-1,[1],[2]]
        ]
      }
    }
  ],
  "objective": {
    "set": "inf",
    "polynomial": {
      "coeftype": "Int64",
      "terms": [
        [1,[4],[1]],
        [1,[2,2]],
        [-1,[3],[2]]
      ]
    }
  },
  "version": "0.0.1",
  "uuid": "dedf3ace-0a05-11eb-172f-49332467ce3a",
  "doc": "A first polynomial example with two sign constraints and one equality constraint.\n"
}


## Moment data

In [4]:
using DynamicPolynomials, PMO
o1 = x^2*y^2+x^4-y^3
o2 = x*y

g1  = x^2 + Float64(pi)*y^2 -2
g2 = x

h1 = 2*y^2-y
h2 = x^2+y*2.1*x*y

F  = PMO.moment(([o1,o2],"inf"),
                ([g1,0],">=0"),
                ([0,g2], ">=0"),
                ([h1, h2], "=0 *")
                )

Optimisation model:
  type => moment
  variables => ["x", "y"]
  nvar => 2
  nms => 2
  constraints => PMO.MomentCstr{Any}(Any[(Polynomial{true,Float64}[x² + 3.141592653589793y² - 2.0, 0.0], ">=0"), (Polynomial{true,Int64}[0, x], ">=0"), (Polynomial{true,Float64}[2.0y² - y, 2.1xy² + x²], "=0 *")], PolyVar{true}[x, y], 2)
  objective => PMO.MomentObj{Array{Polynomial{true,Int64},1}}(Polynomial{true,Int64}[x⁴ + x²y² - y³, xy], "inf", PolyVar{true}[x, y])
  version => 0.0.1
  uuid => e2033e26-0a05-11eb-06ee-e1de01976276


In [5]:
PMO.save("tmp.json",F)
G  = PMO.read("tmp.json")
PMO.json(G)

{
  "type": "moment",
  "variables": [
    "x",
    "y"
  ],
  "nvar": 2,
  "nms": 2,
  "constraints": [
    {
      "set": ">=0",
      "moments": {
        "coeftype": "Float64",
        "terms": [
          [1.0,1,[2],[1]],
          [3.141592653589793,1,[2],[2]],
          [-2.0,1]
        ]
      }
    },
    {
      "set": ">=0",
      "moments": {
        "coeftype": "Int64",
        "terms": [
          [1,2,[1],[1]]
        ]
      }
    },
    {
      "set": "=0 *",
      "moments": {
        "coeftype": "Float64",
        "terms": [
          [2.0,1,[2],[2]],
          [-1.0,1,[1],[2]],
          [2.1,2,[1,2]],
          [1.0,2,[2],[1]]
        ]
      }
    }
  ],
  "objective": {
    "set": "inf",
    "moments": {
      "coeftype": "Int64",
      "terms": [
        [1,1,[4],[1]],
        [1,1,[2,2]],
        [-1,1,[3],[2]],
        [1,2,[1,1]]
      ]
    }
  },
  "version": "0.0.1",
  "uuid": "e2033e26-0a05-11eb-06ee-e1de01976276"
}


## SDP data

In [6]:
using PMO, LinearAlgebra

LMI1 = [Symmetric([2 -1 0; 0 2 0; 0 0 2]),
        0,
        Symmetric([2 0 -1; 0 2 0; 0 0 2])
       ]

LMI2 = [Symmetric([1 0; 0 -1]),
        Symmetric([0 3; 3 0 ]),
        0,
        Symmetric([0 -1; -1 2])
       ]

F  = PMO.sdp(([1,2,3], "inf"),
             (LMI1,">=0"),
             (LMI2,">=0"),
             ([1.1,2,0,-4], "=0"),
             ([0,-1.2,3,-1],"<=0"),
             )
F["name"] = "My first example"
F["doc"]  = "Two linear matrix inequalities, one linear scalar equality and one linear scalar inequality."

"Two linear matrix inequalities, one linear scalar equality and one linear scalar inequality."

In [7]:
PMO.save("tmp.json",F)
G  = PMO.read("tmp.json")
PMO.json(G)

{
  "type": "sdp",
  "nvar": 3,
  "objective": [1,2,3],
  "constraints": {
    "nlmi": 2,
    "msizes": [3,2],
    "lmi_symat": [
      [2.0,1,1,1,1],
      [-1.0,1,1,2,1],
      [2.0,1,1,2,2],
      [2.0,1,1,3,3],
      [2.0,3,1,1,1],
      [2.0,3,1,2,2],
      [-1.0,3,1,3,1],
      [2.0,3,1,3,3],
      [1.0,1,2,1,1],
      [-1.0,1,2,2,2],
      [3.0,2,2,2,1],
      [-1.0,0,2,2,1],
      [2.0,0,2,2,2]
    ],
    "nlsi": 2,
    "lsi_mat": [
      [1.1,1,1],
      [2.0,1,2],
      [1.2,2,2],
      [-3.0,2,3]
    ],
    "lsi_vec": [4.0,-1.0],
    "lsi_op": [0,1]
  },
  "version": "0.0.1",
  "uuid": "e360e5c0-0a05-11eb-301a-41b197aafac2",
  "name": "My first example",
  "doc": "Two linear matrix inequalities, one linear scalar equality and one linear scalar inequality."
}


In [8]:
LMI1 = [Symmetric([2 1 0; 0 1 0; 0 0 0]),
        Symmetric([0 0 0; 0 1 1; 0 0 1]),
        [[0,0,1]],
        -Symmetric([0 0 0; 0 0 1; 0 0 0])]

F  = PMO.sdp(([0,0,1], "inf"),
             (LMI1, ">=0"),
             ([1, 1, 0 , -1], "=0"),
             ([1, 0, 0 ], ">=0"),
             ([0, 1, 0 ], ">=0")
             )
F["name"] = "A second example"
F["doc"] =
    """
    one LMI with one rank-1 matrix, 3 linear scalar constraints
    """

"one LMI with one rank-1 matrix, 3 linear scalar constraints\n"

In [9]:
PMO.save("tmp.json",F)
G  = PMO.read("tmp.json")
PMO.json(G)

{
  "type": "sdp",
  "nvar": 3,
  "objective": [0,0,1],
  "constraints": {
    "nlmi": 1,
    "msizes": 3,
    "lmi_symat": [
      [2.0,1,1,1,1],
      [1.0,1,1,2,1],
      [1.0,1,1,2,2],
      [1.0,2,1,2,2],
      [1.0,2,1,3,2],
      [1.0,2,1,3,3],
      [-1.0,0,1,3,2]
    ],
    "lmi_lrmat": [
      [1.0,3,1,1,3]
    ],
    "nlsi": 3,
    "lsi_mat": [
      [1.0,1,1],
      [1.0,1,2],
      [1.0,2,1],
      [1.0,3,2]
    ],
    "lsi_vec": [1.0,0.0,0.0],
    "lsi_op": [0,1,1]
  },
  "version": "0.0.1",
  "uuid": "e3f68b02-0a05-11eb-36d4-e332b1144430",
  "name": "A second example",
  "doc": "one LMI with one rank-1 matrix, 3 linear scalar constraints\n"
}


## Registering and reading data

We construct the Moztkin polynomial and register a Polynomial data which objective function is Motzkin polynomial.

In [10]:
X = @polyvar x y z
motz = x^4*y^2 + x^2*y^4 + z^6 - 3*x^2*y^2*z^2

x⁴y² + x²y⁴ - 3x²y²z² + z⁶

We define a Polynomial Moment Optimization Problem, with Motzkin homogeneous polynomial minimized on the sphere.

In [11]:
Motz = PMO.polynomial((motz,"inf"), (x^2+y^2+z^2-1, "=0"))
Motz[:name] = "Motzkin homogeneous"

"Motzkin homogeneous"

Now we register it, in a file named `Motzkin.json`

In [12]:
u, file = register(Motz, file="motzkin_homogeneous")

┌ Info: PMO pull data
└ @ PMO /Users/mourrain/Julia/PMO.jl/src/register.jl:39
┌ Info: PMO add data file pmo/motzkin_homogeneous.json
└ @ PMO /Users/mourrain/Julia/PMO.jl/src/register.jl:66
┌ Info: PMO update data
└ @ PMO /Users/mourrain/Julia/PMO.jl/src/register.jl:67
┌ Info: PMO register pmo/motzkin_homogeneous.json
└ @ PMO /Users/mourrain/Julia/PMO.jl/src/register.jl:121
┌ Info: PMO update registries
└ @ PMO /Users/mourrain/Julia/PMO.jl/src/register.jl:98


(UUID("e5c4d450-0a05-11eb-25c5-3d1bbc34e4c3"), "motzkin_homogeneous")

The data is stored in data project `PMO_GIT_DATA_URL` as the file `pmo/motzkin.json`. The index tables of the registry project `PMO_GIT_REGISTRY_URL`are update with a line per data. A local version of the data bases and index files are updated and wWe use `git` command to commit and push the changes on the `git`projects.

The tables in the registery project are available with the function `gettable`. The default table is a table of triplets (uuid , url of the data, url of the documentation):

In [13]:
pmodb = pmotable()

PMO.DataBase(Table with 8 rows, 3 columns:
Columns:
#  colname  type
──────────────────
1  uuid     String
2  name     String
3  url      String)

We can recover the i$^{th}$ entry of this database:

In [14]:
pmodb[1]

Optimisation model:
  type => polynomial
  name => Example0
  nvar => 2
  variables => Any["x", "y"]
  objective => PMO.PolynomialObj{Polynomial{true,Int64}}(x⁴ + x²y² - y³, "inf", PolyVar{true}[x, y])
  constraints => PMO.PolynomialCstr{Any}(Any[(x² + 3.141592653589793y² - 2.0, "<=0"), (x, Any[-1, 1]), (2y² - y, "=0")], PolyVar{true}[x, y])
  version => 0.0.1
  author => Bernard Mourrain
  uuid => 83c94e7a-f3ab-5447-9af7-d94c0b1920cf


We can recover entries from regular expressions matching the name entries:

In [ ]:
pmodb[r"Motz"]